In [2]:
import os
import numpy as np
import typing as ty
import lava

# Import Process level primitives
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort

# Import parent classes for ProcessModels
from lava.magma.core.model.sub.model import AbstractSubProcessModel
from lava.magma.core.model.py.model import PyLoihiProcessModel

# Import ProcessModel ports, data-types
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.model.py.type import LavaPyType

# Import execution protocol and hardware resources
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.resources import CPU

# Import decorators
from lava.magma.core.decorator import implements, requires

# Import MNIST dataset
from lava.utils.dataloader.mnist import MnistDataset
np.set_printoptions(linewidth=np.inf)

from dataset import Dataset, SpikingDataset, RegSpikingDataset
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense  
%load_ext autoreload

In [3]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [1000]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S2'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23c
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]
i = 1
rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
        N_reservoir=16,
        spectral_radius=0.2,
        sparsity=0.4,
        noise=1e-6,
        lut_activation=False,  # True,
        tanh_lut=tanh_lut,
        input_scale=25,  #40, #50, # 25,
        reservoir_input_scale = 8,  #4,  #5,
        show_wout=False,
        output_folder= output_folder,
        debug=debug,
        use_fpga= None,
        sock= None,  # usock
        addr = None) # addr

train_input, train_label, test_input, test_label = rc.run()

In [4]:
sequence_length = 2
num_input_node = 4
num_steps = 100
train_data = RegSpikingDataset(train_input, train_label, sequence_length, num_steps)
dataset = []
for i in range(len(train_data)):
    dataset.append(train_data[i][0].reshape((num_steps, -1)).astype(bool))

In [5]:
import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(dataset, fp)

In [12]:
def Quantization(path):
    weights = []
    f = open(path, 'rb')
    mask1 = np.load(f)
    mask2 = np.load(f)
    weight1 = np.load(f)
    weights.append(mask1)
    weights.append(mask2)
    weights.append(weight1)
    f.close()

    print(mask1.shape, mask2.shape, weight1.shape)

    # Quantize weights and biases using max-normalization (Strong quantization loss if distributions have large tails)
    from nxsdk_modules_ncl.dnn.src.utils import to_integer
    p_int = []
    Vth, int_w = [], []

    for w in weights:
        #length = np.n([np.abs(np.min(w)), np.abs(np.max(w))])
        print(np.min(w), np.max(w))
        length = np.max([np.abs(np.min(w)), np.abs(np.max(w))])
        print(np.round(255.0/length))
        Vth.append(np.round(255.0/length))
        b = np.zeros(w.shape[0])
        w_int, b_int = to_integer(w, b, 8)

        int_w.append(w_int)

    return int_w, Vth


class SpikeInput(AbstractProcess):
    """Reads image data from the MNIST dataset and converts it to spikes.
    The resulting spike rate is proportional to the pixel value."""

    def __init__(self,
                 sequence_length : ty.Optional[int] = 2,
                 num_input_node : ty.Optional[int] = 4,
                 num_steps: ty.Optional[int] = 100):
        super().__init__()
        self.sequence_length = Var(shape=(1,), init=sequence_length)
        self.num_input_node = Var(shape=(1,), init=num_input_node) 
        shape = (sequence_length * num_input_node,)
        self.spikes_out = OutPort(shape=shape)
        self.num_steps = Var(shape=(1,), init=num_steps)
        self.input_data = Var(shape=shape)
            
        
@implements(proc=SpikeInput, protocol=LoihiProtocol)
@requires(CPU)
class PySpikeInputModel(PyLoihiProcessModel):
    spikes_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, bool, precision=1)
    input_data: np.ndarray = LavaPyType(np.ndarray, bool, precision=1)
    num_steps: int = LavaPyType(int, int, precision=32)
    sequence_length: int = LavaPyType(int, int, precision=32)
    num_input_node: int = LavaPyType(int, int, precision=32)
    
    def __init__(self, proc_params):
        super().__init__(proc_params=proc_params)
        self.dataset = dataset
        self.curr_id = 0
        from collections import defaultdict
        self.mp = defaultdict(int)
        self.cnt = 0

    def post_guard(self):
        """Guard function for PostManagement phase.
        """
        if self.time_step % 1 == 0:
            return True
        return False
        #return True

    def run_post_mgmt(self):
        """Post-Management phase: executed only when guard function above 
        returns True.
        """
        # time_step start with 1
        curr_idx = (self.time_step - 1) // self.num_steps
        cur_time_step = (self.time_step - 1) % self.num_steps
        self.input_data = self.dataset[self.curr_id][cur_time_step, :]
    
    def run_spk(self):
        """Spiking phase: executed unconditionally at every time-step
        """
        print(self.input_data)
        self.spikes_out.send(self.input_data)
        #print(self.time_step)
        #print(self.input_data)
        #print(self.dataset[self.curr_id][(self.time_step - 1) % (self.num_steps)])
               


In [14]:
from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg

sequence_length = 2
num_input_node = 4
num_steps = 100
train_data = RegSpikingDataset(train_input, train_label, sequence_length, num_steps)
abc = [False,  True, False, False, False,  True, False, False]
for i, row in enumerate(dataset[1]):
    cnt = 0
    for j, val in enumerate(row):
        if val == abc[j]:
            cnt += 1
        else:
            break
    if cnt == 8:
        print("adsfdasf", i)

spike_input = SpikeInput(
                         sequence_length,
                         num_input_node,
                         num_steps)


dense = Dense(weights=125 * np.ones((8, 8)))
spike_input.spikes_out.connect(dense.s_in)
#layer1.spikes.get()


for idx in range(2):
    print("cur_idx", idx)
    dense.run(
        condition=RunSteps(num_steps=num_steps),
        run_cfg=Loihi1SimCfg(select_sub_proc_model=True,
                             select_tag='fixed_pt'))
    x = spike_input.input_data.get().astype(bool)
    print(x.shape)
    #spike_input.input_data.set(np.zeros((8,), dtype=bool))
    #print()
    #print(x)
    #print(dataset[idx + 1][0, :])

dense.stop()
    #ground_truth = output_proc.gt_labels.get().astype(np.int32)
    #predictions = output_proc.pred_labels.get().astype(np.int32)

adsfdasf 50
cur_idx 0
[False False False False False False False False]
[ True  True False  True False  True  True False]
[ True  True False  True  True False  True  True]
[ True  True False  True  True False  True  True]
[ True  True  True False False  True  True False]
[ True False False  True False  True  True False]
[False False  True  True False  True False  True]
[False  True  True False False False False False]
[False False False False False  True  True  True]
[ True False  True False  True  True False  True]
[ True False  True False  True  True  True  True]
[False  True False False False  True  True False]
[False  True  True False  True False  True False]
[False  True  True False False  True False  True]
[ True  True  True  True False  True False  True]
[False False False False  True  True  True False]
[ True  True False  True  True  True False  True]
[ True  True False  True  True False False  True]
[False  True  True False  True  True  True  True]
[False False False  True  Tr

In [11]:
class RegModel(AbstractProcess):
    """A 3 layer feed-forward network with LIF and Dense Processes."""

    def __init__(self, path: str):
        super().__init__()
        
        # Using pre-trained weights and biases
        weights = []
        f = open(path, 'rb')
        w0 = np.load(f)
        w1 = np.load(f)
        w2 = np.load(f)
        f.close()

        self.spikes_in = InPort(shape=(w0.shape[1],))
        #self.spikes_out = OutPort(shape=(w2.shape[0],))
        self.w_dense0 = Var(shape=w0.shape, init=w0)
        self.w_dense1 = Var(shape=w1.shape, init=w1)
        self.w_dense2 = Var(shape=w2.shape, init=w2)
        
        # Up-level currents and voltages of LIF Processes
        # for resetting (see at the end of the tutorial)
        self.lif1_u = Var(shape=(w0.shape[0],), init=0)
        self.lif1_v = Var(shape=(w0.shape[0],), init=0)
        self.lif2_u = Var(shape=(w1.shape[0],), init=0)
        self.lif2_v = Var(shape=(w1.shape[0],), init=0)
        #self.oplif_u = Var(shape=(w2.shape[0],), init=0)
        #self.oplif_v = Var(shape=(w2.shape[0],), init=0)
        
        
class OutputProcess(AbstractProcess):
    """Process to gather spikes from 10 output LIF neurons and interpret the
    highest spiking rate as the classifier output"""

    def __init__(self, **kwargs):
        super().__init__()
        shape = (10,)
        n_img = kwargs.pop('num_images', 25)
        self.num_images = Var(shape=(1,), init=n_img)
        self.spikes_in = InPort(shape=shape)
        self.label_in = InPort(shape=(1,))
        self.spikes_accum = Var(shape=shape)  # Accumulated spikes for classification
        self.num_steps_per_image = Var(shape=(1,), init=128)
        self.pred_labels = Var(shape=(n_img,))
        self.gt_labels = Var(shape=(n_img,))

In [6]:
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense            

@implements(ImageClassifier)
@requires(CPU)
class PyImageClassifierModel(AbstractSubProcessModel):
    def __init__(self, proc):
        self.dense0 = Dense(weights=proc.w_dense0.init)
        self.lif1 = LIF(shape=(64,), bias_mant=proc.b_lif1.init, vth=400,
                        dv=0, du=4095)
        self.dense1 = Dense(weights=proc.w_dense1.init)
        self.lif2 = LIF(shape=(64,), bias_mant=proc.b_lif2.init, vth=350,
                        dv=0, du=4095)
        self.dense2 = Dense(weights=proc.w_dense2.init)
        self.output_lif = LIF(shape=(10,), bias_mant=proc.b_output_lif.init,
                              vth=1, dv=0, du=4095)

        proc.spikes_in.connect(self.dense0.s_in)
        self.dense0.a_out.connect(self.lif1.a_in)
        self.lif1.s_out.connect(self.dense1.s_in)
        self.dense1.a_out.connect(self.lif2.a_in)
        self.lif2.s_out.connect(self.dense2.s_in)
        self.dense2.a_out.connect(self.output_lif.a_in)
        self.output_lif.s_out.connect(proc.spikes_out)
        
        # Create aliases of SubProcess variables
        proc.lif1_u.alias(self.lif1.u)
        proc.lif1_v.alias(self.lif1.v)
        proc.lif2_u.alias(self.lif2.u)
        proc.lif2_v.alias(self.lif2.v)
        proc.oplif_u.alias(self.output_lif.u)
        proc.oplif_v.alias(self.output_lif.v)

In [7]:
@implements(proc=OutputProcess, protocol=LoihiProtocol)
@requires(CPU)
class PyOutputProcessModel(PyLoihiProcessModel):
    label_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, int, precision=32)
    spikes_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, bool, precision=1)
    num_images: int = LavaPyType(int, int, precision=32)
    spikes_accum: np.ndarray = LavaPyType(np.ndarray, np.int32, precision=32)
    num_steps_per_image: int = LavaPyType(int, int, precision=32)
    pred_labels: np.ndarray = LavaPyType(np.ndarray, int, precision=32)
    gt_labels: np.ndarray = LavaPyType(np.ndarray, int, precision=32)
        
    def __init__(self, proc_params):
        super().__init__(proc_params=proc_params)
        self.current_img_id = 0

    def post_guard(self):
        """Guard function for PostManagement phase.
        """
        if self.time_step % self.num_steps_per_image == 0 and \
                self.time_step > 1:
            return True
        return False

    def run_post_mgmt(self):
        """Post-Management phase: executed only when guard function above 
        returns True.
        """
        gt_label = self.label_in.recv()
        pred_label = np.argmax(self.spikes_accum)
        self.gt_labels[self.current_img_id] = gt_label
        self.pred_labels[self.current_img_id] = pred_label
        self.current_img_id += 1
        self.spikes_accum = np.zeros_like(self.spikes_accum)

    def run_spk(self):
        """Spiking phase: executed unconditionally at every time-step
        """
        spk_in = self.spikes_in.recv()
        self.spikes_accum = self.spikes_accum + spk_in